In [96]:
import pandas as pd
import numpy as np

In [97]:
df = pd.read_csv('diabetes.csv')

In [98]:
df.head(4)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0


In [99]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [100]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [101]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [102]:
X = scaler.fit_transform(X)

In [103]:
X.shape

(768, 8)

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1)

In [105]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Dropout


In [106]:
model = Sequential()

model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

/Users/mdalamin/anaconda3/envs/LOUDA_LASCHAN/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [107]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [108]:
model.fit(X_train, y_train, batch_size=32, epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.4105 - loss: 0.7840 - val_accuracy: 0.4286 - val_loss: 0.7524
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4832 - loss: 0.7131 - val_accuracy: 0.5779 - val_loss: 0.6812
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5921 - loss: 0.6504 - val_accuracy: 0.6818 - val_loss: 0.6275
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6858 - loss: 0.6050 - val_accuracy: 0.7338 - val_loss: 0.5857
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6999 - loss: 0.5869 - val_accuracy: 0.7597 - val_loss: 0.5560
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7309 - loss: 0.5561 - val_accuracy: 0.7662 - val_loss: 0.5338
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7322 - loss: 0.5497 - val_accuracy: 0.7662 - val_loss: 0.5184
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7823 - loss: 0.5008 - val_accuracy: 0.7727 - 

In [109]:
# 1. How to select appropriate optimizer
# 2. No. of nodes or neuron in a layer
# 3. How to select no. of layers
# 4. All in all one model

In [110]:
import kerastuner as kt

In [111]:
def build_model(hp):
    model = Sequential()
    
    model.add(Dense(32, activation='relu', input_dim=8))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = hp.Choice('optimizer', values = ['rmsprop', 'sgd', 'adam', 'adadelta'])
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [112]:
tuner = kt.RandomSearch(build_model, 
                        objective='val_accuracy',
                        max_trials=5)

In [113]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.6623376607894897

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 08s


In [114]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [115]:
model = tuner.get_best_models(num_models=1)[0]

/Users/mdalamin/anaconda3/envs/LOUDA_LASCHAN/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [116]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [117]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=[X_test, y_test])

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7578 - loss: 0.5215 - val_accuracy: 0.7922 - val_loss: 0.4946
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7554 - loss: 0.4994 - val_accuracy: 0.7922 - val_loss: 0.4860
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7682 - loss: 0.4839 - val_accuracy: 0.7922 - val_loss: 0.4800
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7724 - loss: 0.4639 - val_accuracy: 0.7857 - val_loss: 0.4753
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7771 - loss: 0.4573 - val_accuracy: 0.7857 - val_loss: 0.4716
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7610 - loss: 0.4724 - val_accuracy: 0.7987 - val_loss: 0.4683
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7560 - loss: 0.4648 - val_accuracy: 0.8052 - val_loss: 0.4667
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7709 - loss: 0.4721 - val_accuracy: 0.805

## Find No of Neuron

In [118]:
def build_model(hp):
    model = Sequential()
    
    unints = hp.Int('units', min_value = 8,max_value = 128, step = 8)    

    model.add(Dense(units=unints, activation='relu', input_dim = 8))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [119]:
tuner = kt.RandomSearch(build_model, 
                        objective='val_accuracy',
                        max_trials=10,
                        directory = 'mydir',
                        project_name = 'hp_tuning'
                        )

In [120]:
tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.6623376607894897

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 18s


In [121]:
tuner.get_best_hyperparameters()[0].values

{'units': 64}

In [122]:
model = tuner.get_best_models(num_models=1)[0]

In [123]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.7514 - loss: 0.5248 - val_accuracy: 0.7922 - val_loss: 0.4923
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7815 - loss: 0.4999 - val_accuracy: 0.7987 - val_loss: 0.4799
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7842 - loss: 0.4689 - val_accuracy: 0.7857 - val_loss: 0.4761
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7702 - loss: 0.4853 - val_accuracy: 0.7922 - val_loss: 0.4713
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7772 - loss: 0.4588 - val_accuracy: 0.8052 - val_loss: 0.4674
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7819 - loss: 0.4594 - val_accuracy: 0.7987 - val_loss: 0.4659
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7927 - loss: 0.4441 - val_accuracy: 0.7857 - val_loss: 0.4634
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7790 - loss: 0.4439 - val_accuracy: 0.798

## How to select No of layers

In [124]:
def build_model(hp):
    model = Sequential()
    
    model.add(Dense(104, activation='relu', input_dim = 8))
    
    for i in range(hp.Int('num_layers', min_value=1, max_value = 10)):
        model.add(Dense(104, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [125]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name = 'hp_tuning_layers' 
                        )

In [126]:
# tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

tuner.search(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8116883039474487

Best val_accuracy So Far: 0.8246753215789795
Total elapsed time: 00h 00m 13s


In [127]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1}

In [128]:
model = tuner.get_best_models(num_models=1)[0]

/Users/mdalamin/anaconda3/envs/LOUDA_LASCHAN/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 8 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [129]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 104)            │           936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 104)            │        10,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           105 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,961 (46.72 KB)

 Trainable params: 11,961 (46.72 KB)

 Non-trainable params: 0 (0.00 B)

In [130]:
model.fit(X_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8172 - loss: 0.4290 - val_accuracy: 0.7922 - val_loss: 0.4702
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7823 - loss: 0.4565 - val_accuracy: 0.7987 - val_loss: 0.4620
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7735 - loss: 0.4532 - val_accuracy: 0.7857 - val_loss: 0.4732
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7848 - loss: 0.4468 - val_accuracy: 0.7792 - val_loss: 0.4833
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7876 - loss: 0.4431 - val_accuracy: 0.7792 - val_loss: 0.4779
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7792 - loss: 0.4271 - val_accuracy: 0.7987 - val_loss: 0.4596
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7834 - loss: 0.4338 - val_accuracy: 0.7597 - val_loss: 0.4704
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8033 - loss: 0.4236 - val_accuracy: 0.78

## All in One Code

In [134]:
def build_model(hp):
    model = Sequential()
    
    counter = 0
    
    for i in range(hp.Int('num_layers', min_value=1, max_value=15)):
        if counter == 0:
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation' + str(i), values = ['relu', 'tanh', 'sigmoid', 'softmax']),
                    input_dim = 8
                )
            )
            model.add(Dropout(hp.Choice('dropout', values=[0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9])))
        else:
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value=8, max_value=128, step=8),
                    activation=hp.Choice('activation' + str(i), values = ['relu', 'tanh', 'sigmoid', 'softmax'])
                )
            )
            model.add(Dropout(hp.Choice('dropout', values=[0.1, 0.2, 0.3, 0.4, 0.6, 0.7, 0.8, 0.9])))
        counter += 1
    
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop', 'adam', 'sgd']), 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

        

In [136]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=5,
                        directory = 'mydir',
                        project_name = 'final_droput'
                        )

In [137]:
tuner.search(X_train, y_train, epochs=10, validation_data = (X_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.701298713684082

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 19s


In [138]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 112,
 'activation0': 'sigmoid',
 'dropout': 0.2,
 'optimizer': 'rmsprop',
 'units1': 104,
 'activation1': 'relu',
 'units2': 88,
 'activation2': 'softmax',
 'units3': 128,
 'activation3': 'sigmoid',
 'units4': 48,
 'activation4': 'relu',
 'units5': 72,
 'activation5': 'relu',
 'units6': 24,
 'activation6': 'sigmoid',
 'units7': 32,
 'activation7': 'relu',
 'units8': 112,
 'activation8': 'tanh',
 'units9': 40,
 'activation9': 'tanh',
 'units10': 80,
 'activation10': 'tanh',
 'units11': 40,
 'activation11': 'sigmoid',
 'units12': 80,
 'activation12': 'tanh',
 'units13': 72,
 'activation13': 'relu'}

In [139]:
model = tuner.get_best_models(num_models=1)[0]

In [140]:
model.fit(X_train, y_train, epochs=200, initial_epoch=10, validation_data=(X_test, y_test))

Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7492 - loss: 0.5191 - val_accuracy: 0.8117 - val_loss: 0.4840
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7675 - loss: 0.4713 - val_accuracy: 0.8182 - val_loss: 0.4846
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7668 - loss: 0.4914 - val_accuracy: 0.7468 - val_loss: 0.5492
Epoch 14/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7601 - loss: 0.5415 - val_accuracy: 0.7857 - val_loss: 0.4694
Epoch 15/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7356 - loss: 0.5099 - val_accuracy: 0.7987 - val_loss: 0.4731
Epoch 16/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7537 - loss: 0.5141 - val_accuracy: 0.8117 - val_loss: 0.4930
Epoch 17/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7825 - loss: 0.4635 - val_accuracy: 0.7857 - val_loss: 0.4683
Epoch 18/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7342 - loss: 0.5107 - val_accuracy: 0

In [141]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 112)            │         1,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 104)            │        11,752 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 104)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           105 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,732 (100.52 KB)

 Trainable params: 12,865 (50.25 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 12,867 (50.27 KB)